In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.float_format", "{:,.2f}".format)
from tabulate import tabulate
rho_value = "08"
encoding = "enc0"

To be adjusted for the calculation of the Sub-Questions and Response Variables.

In [2]:
# if encoding is not None:
#     table_source = f"../logs/smartRPA/202511-results/validation_experiment_results_word2vec_safety2_olap08_rho{rho_value}_{encoding}.csv"
# else:
#     table_source = f"../logs/smartRPA/202511-results/validation_experiment_results_word2vec_safety2_olap08_rho{rho_value}.csv"
table_source = f"../logs/smartRPA/202511-results/validation_experiment_results_word2vec_safety2_olap08_rho{rho_value}_enc0_core08.csv"
df = pd.read_csv(table_source)
df.rename(columns={"motif0-length":"motLength","motif0-occurances":"motOcc","motif0-shuffle":"motShuffle"}, inplace=True)

rho_value_id = "rho" + rho_value

In [3]:
def calculate_classification_metrics(df):
    # Create a copy to avoid SettingWithCopy warnings if the input is a slice
    res_df = df.copy()

    # 1. Recall: TP / (TP + FN)
    # Measures: Out of all the actual motifs in the data, how many did we find?
    res_df["recall"] = res_df["total_tp"] / (res_df["total_tp"] + res_df["total_fn"])

    # 2. Precision: TP / (TP + FP)
    # Measures: Out of all the motifs we claimed to find, how many were actually correct?
    res_df["precision"] = res_df["total_tp"] / (res_df["total_tp"] + res_df["total_fp"])

    # 3. F1-score: Harmonic mean of Precision and Recall
    # Measures: Balance between Precision and Recall.
    res_df["fscore"] = (2 * (res_df["precision"] * res_df["recall"]) / 
                          (res_df["precision"] + res_df["recall"]))

    # Optional: Fill NaN values with 0.0 (happens if denominator is 0)
    res_df[["recall", "precision", "fscore"]] = res_df[["recall", "precision", "fscore"]].fillna(0.0)
    return res_df

## Results for all Logs of EX2 based on rho_value

In [4]:
print("Results from file:")
print(table_source)
print("\n")

pivotPercentage = pd.pivot_table(df, index=["percentageMotifsOverLog"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
# pivotLogLength  = pd.pivot_table(df, index=["logLength"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotNoOfMotifs = pd.pivot_table(df, index=["noOfMotifs"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotOcc     = pd.pivot_table(df, index=["motOcc"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotLength  = pd.pivot_table(df, index=["motLength"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotShuffle = pd.pivot_table(df, index=["motShuffle"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')

list_of_pivots = [pivotPercentage, pivotNoOfMotifs, pivotMotOcc, pivotMotLength, pivotMotShuffle]

for pivot in list_of_pivots: 
    pivot = calculate_classification_metrics(pivot)
    # Assuming 'final_results' is your DataFrame
    print(tabulate(pivot, headers='keys', tablefmt='psql', numalign="right"))

Results from file:
../logs/smartRPA/202511-results/validation_experiment_results_word2vec_safety2_olap08_rho08_enc0_core08.csv


+---------------------------+------------+------------+------------+----------+-------------+----------+
|   percentageMotifsOverLog |   total_fn |   total_fp |   total_tp |   recall |   precision |   fscore |
|---------------------------+------------+------------+------------+----------+-------------+----------|
|                         1 |          0 |       2107 |         45 |        1 |   0.0209108 | 0.040965 |
|                        10 |         23 |       7191 |       1867 | 0.987831 |    0.206116 | 0.341067 |
|                        25 |        102 |        568 |       2593 | 0.962152 |     0.82031 | 0.885587 |
|                        50 |        496 |        438 |       2474 | 0.832997 |    0.849588 |  0.84121 |
|                        75 |        764 |        487 |       2206 | 0.742761 |    0.819161 | 0.779092 |
|                       100 |  

## Result Pivots for Filtered 1% and 100% of EX2 based on rho_value

In [5]:
print("Results from file:")
print(table_source)
print("\n")

dfFilteredOne = df[~df["percentageMotifsOverLog"].isin([1.00,100.00])]
pivotPercentageFiltered = pd.pivot_table(dfFilteredOne, index=["percentageMotifsOverLog"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
# pivotLogLengthFiltered  = pd.pivot_table(dfFilteredOne, index=["logLength"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotNoOfMotifsFiltered = pd.pivot_table(dfFilteredOne, index=["noOfMotifs"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotOccFiltered     = pd.pivot_table(dfFilteredOne, index=["motOcc"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotLengthFiltered  = pd.pivot_table(dfFilteredOne, index=["motLength"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotShuffleFiltered = pd.pivot_table(dfFilteredOne, index=["motShuffle"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')

list_of_filtered_pivots = [pivotPercentageFiltered, pivotNoOfMotifsFiltered, pivotMotOccFiltered, pivotMotLengthFiltered, pivotMotShuffleFiltered]

for pivot in list_of_filtered_pivots:   
    pivot = calculate_classification_metrics(pivot)
    # 3. (Optional) Keep your visual print if you still want to see it in the console
    print(tabulate(pivot, headers='keys', tablefmt='psql', numalign="right"))

Results from file:
../logs/smartRPA/202511-results/validation_experiment_results_word2vec_safety2_olap08_rho08_enc0_core08.csv


+---------------------------+------------+------------+------------+----------+-------------+----------+
|   percentageMotifsOverLog |   total_fn |   total_fp |   total_tp |   recall |   precision |   fscore |
|---------------------------+------------+------------+------------+----------+-------------+----------|
|                        10 |         23 |       7191 |       1867 | 0.987831 |    0.206116 | 0.341067 |
|                        25 |        102 |        568 |       2593 | 0.962152 |     0.82031 | 0.885587 |
|                        50 |        496 |        438 |       2474 | 0.832997 |    0.849588 |  0.84121 |
|                        75 |        764 |        487 |       2206 | 0.742761 |    0.819161 | 0.779092 |
+---------------------------+------------+------------+------------+----------+-------------+----------+
+--------------+------------+--

## Latex Code to be copied into the paper for results (All rho)

Note: Hard Coded Names of Validation Files!

In [6]:
# ==========================================
# 1. HELPER: Calculate Metrics (Renamed f1score -> fscore)
# ==========================================
def calculate_classification_metrics(df):
    """
    Calculates Precision, Recall, and F-Score.
    Column names generated: 'recall', 'precision', 'fscore'.
    """
    # Avoid division by zero
    df['recall'] = df.apply(lambda row: row['total_tp'] / (row['total_tp'] + row['total_fn']) 
                            if (row['total_tp'] + row['total_fn']) > 0 else 0, axis=1)
    
    df['precision'] = df.apply(lambda row: row['total_tp'] / (row['total_tp'] + row['total_fp']) 
                               if (row['total_tp'] + row['total_fp']) > 0 else 0, axis=1)
    
    # Renamed to 'fscore' to avoid numbers in LaTeX variables
    df['fscore'] = df.apply(lambda row: 2 * (row['precision'] * row['recall']) / (row['precision'] + row['recall']) 
                             if (row['precision'] + row['recall']) > 0 else 0, axis=1)
    return df

# ==========================================
# 2. HELPER: Smart Latex Name Sanitizer
# ==========================================
def sanitize_latex_name(text):
    """
    Converts keys like '09filtered' -> 'ZeroNineFiltered'
    and 'rho06' -> 'RhoZeroSix'.
    """
    clean = str(text)
    
    # 1. Specific fix to ensure 'filtered' becomes 'Filtered' (CamelCase)
    if "filtered" in clean:
        clean = clean.replace("filtered", "Filtered")

    # 2. Map numbers to TitleCase words
    num_map = {
        '0': 'Zero', '1': 'One', '2': 'Two', '3': 'Three', '4': 'Four',
        '5': 'Five', '6': 'Six', '7': 'Seven', '8': 'Eight', '9': 'Nine',
        '.': 'Point'
    }
    for char, word in num_map.items():
        clean = clean.replace(char, word)
    
    # 3. Handle Underscores (Snake_case -> CamelCase)
    if '_' in clean:
        # Capitalize the first letter of each segment
        return "".join(word[0].upper() + word[1:] for word in clean.split('_') if word)
    else:
        # Ensure the very first letter is Capitalized (e.g. fscore -> Fscore)
        return clean[0].upper() + clean[1:]

# ==========================================
# 3. HELPER: Print Pivot Variables
# ==========================================
def print_pivot_variables(df, rho_id):
    metric_name = df.index.name
    # Updated target list to look for 'fscore'
    target_metrics = ['precision', 'recall','fscore']
    
    safe_rho = sanitize_latex_name(rho_id)
    safe_metric_name = sanitize_latex_name(metric_name)

    print(f"% --- {safe_rho} Variables for Index: {metric_name} ---")
    
    for index_val, row in df.iterrows():
        safe_idx = sanitize_latex_name(index_val)
        
        for type_col in target_metrics:
            if type_col in df.columns:
                val = f"{row[type_col]:.3f}"
                safe_type = sanitize_latex_name(type_col) # e.g. fscore -> Fscore
                
                # Name construction: \rhoZeroNineFilteredRecallNoOfMotifsTwo
                var_name = f"{safe_rho}{safe_type}{safe_metric_name}{safe_idx}"
                # Standard convention: first letter lowercase (\rho...)
                var_name = var_name[0].lower() + var_name[1:] 
                
                print(f"\\def\\{var_name}{{{val}}}")
    print("")

# ==========================================
# 4. HELPER: Print Overall Global Variables
# ==========================================
def print_overall_variables(df, rho_id):
    # Sum totals
    total_tp = df["total_tp"].sum()
    total_fp = df["total_fp"].sum()
    total_fn = df["total_fn"].sum()
    
    # Calculate metrics globally
    rec = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    prec = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    f_score = 2 * (prec * rec) / (prec + rec) if (prec + rec) > 0 else 0
    
    # Updated keys to use 'Fscore'
    metrics = {
        'TotalTp': total_tp, 'TotalFp': total_fp, 'TotalFn': total_fn,
        'Recall': rec, 'Precision': prec, 'Fscore': f_score
    }
    
    safe_rho = sanitize_latex_name(rho_id)
    print(f"% --- {safe_rho} Global Metrics ---")
    
    for name, value in metrics.items():
        formatted_val = f"{int(value)}" if "Total" in name else f"{value:.3f}"
        
        # Name construction: \rhoZeroNineFilteredFscoreOverall
        var_name = f"{safe_rho}{name}Overall"
        var_name = var_name[0].lower() + var_name[1:]
        
        print(f"\\def\\{var_name}{{{formatted_val}}}")
    print("")

# ==========================================
# 5. MAIN EXECUTION FUNCTION
# ==========================================
def generate_full_latex_report(data_dictionary):
    """
    Accepts a dictionary of keys ("06", "09filtered") and dataframes.
    Generates Latex \def commands for all of them using 'Fscore'.
    """
    print("% ========================================================")
    print("% AUTOMATED LATEX VARIABLES GENERATION")
    print("% ========================================================")
    print("")
    
    for key, df in data_dictionary.items():
        # Construct ID: "06" -> "rho06", "06filtered" -> "rho06filtered"
        rho_id = f"rho{key}"
        
        # 1. Global Metrics (Overall sums)
        print_overall_variables(df, rho_id)
        
        # 2. Create Pivots
        pivotPercentage = pd.pivot_table(df, index=["percentageMotifsOverLog"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
        pivotNoOfMotifs = pd.pivot_table(df, index=["noOfMotifs"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
        pivotMotOcc     = pd.pivot_table(df, index=["motOcc"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
        pivotMotLength  = pd.pivot_table(df, index=["motLength"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
        pivotMotShuffle = pd.pivot_table(df, index=["motShuffle"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')

        list_of_pivots = [pivotPercentage, pivotNoOfMotifs, pivotMotOcc, pivotMotLength, pivotMotShuffle]
        
        # 3. Process Pivots
        for pivot in list_of_pivots:
            pivot = calculate_classification_metrics(pivot)
            print_pivot_variables(pivot, rho_id)
        
        print(f"% --------------------------------------------------------") 


In [7]:
name_06 = f"../logs/smartRPA/202511-results/validation_experiment_results_word2vec_safety2_olap08_rho06_enc0_core08.csv"
name_07 = f"../logs/smartRPA/202511-results/validation_experiment_results_word2vec_safety2_olap08_rho07_enc0_core08.csv"
name_08 = f"../logs/smartRPA/202511-results/validation_experiment_results_word2vec_safety2_olap08_rho08_enc0_core08.csv"
name_09 = f"../logs/smartRPA/202511-results/validation_experiment_results_word2vec_safety2_olap08_rho09_enc0_core08.csv"
df06 = pd.read_csv(name_06)
df07 = pd.read_csv(name_07)
df08 = pd.read_csv(name_08)
df09 = pd.read_csv(name_09)

df06.rename(columns={"motif0-length":"motLength","motif0-occurances":"motOcc","motif0-shuffle":"motShuffle"}, inplace=True)
df07.rename(columns={"motif0-length":"motLength","motif0-occurances":"motOcc","motif0-shuffle":"motShuffle"}, inplace=True)
df08.rename(columns={"motif0-length":"motLength","motif0-occurances":"motOcc","motif0-shuffle":"motShuffle"}, inplace=True)
df09.rename(columns={"motif0-length":"motLength","motif0-occurances":"motOcc","motif0-shuffle":"motShuffle"}, inplace=True)

df06filtered = df06[~df06["percentageMotifsOverLog"].isin([1.00,100.00])]
df07filtered = df07[~df07["percentageMotifsOverLog"].isin([1.00,100.00])]
df08filtered = df08[~df08["percentageMotifsOverLog"].isin([1.00,100.00])]
df09filtered = df09[~df09["percentageMotifsOverLog"].isin([1.00,100.00])]

# REPLACE THESE WITH YOUR ACTUAL DATAFRAMES
data_map = {
    "06": df06,
    "07": df07,
    "08": df08,
    "09": df09,
    "06filtered": df06filtered,
    "07filtered": df07filtered,
    "08filtered": df08filtered,
    "09filtered": df09filtered
}

# Example call (using your current 'df' as a placeholder for all):
generate_full_latex_report(data_map)

% ========================================================
% AUTOMATED LATEX VARIABLES GENERATION
% ========================================================

% --- RhoZeroSix Global Metrics ---
\def\rhoZeroSixTotalTpOverall{8187}
\def\rhoZeroSixTotalFpOverall{7014}
\def\rhoZeroSixTotalFnOverall{5073}
\def\rhoZeroSixRecallOverall{0.617}
\def\rhoZeroSixPrecisionOverall{0.539}
\def\rhoZeroSixFscoreOverall{0.575}

% --- RhoZeroSix Variables for Index: percentageMotifsOverLog ---
\def\rhoZeroSixPrecisionPercentageMotifsOverLogOnePointZero{0.033}
\def\rhoZeroSixRecallPercentageMotifsOverLogOnePointZero{1.000}
\def\rhoZeroSixFscorePercentageMotifsOverLogOnePointZero{0.064}
\def\rhoZeroSixPrecisionPercentageMotifsOverLogOneZeroPointZero{0.242}
\def\rhoZeroSixRecallPercentageMotifsOverLogOneZeroPointZero{0.957}
\def\rhoZeroSixFscorePercentageMotifsOverLogOneZeroPointZero{0.386}
\def\rhoZeroSixPrecisionPercentageMotifsOverLogTwoFivePointZero{0.849}
\def\rhoZeroSixRecallPercentageMotifsOverLogTwo